In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch as t
from torch import nn
import numpy as np
try:
    import ipdb
except:
    import pdb as ipdb

In [8]:
class Conv2dBlock(nn.Module):
    def __init__(self,ninp,fmaps,kwidth,stride):
        super().__init__()
        self.conv=nn.Conv2d(ninp,fmaps,kwidth,stride)
        self.pool=nn.MaxPool2d(kernel_size=(1,2),stride=(1,2))
        self.act=nn.ELU()
        
    def forward(self,x):
        x=self.conv(x)
        x=self.pool(x)
        x=self.act(x)
        return x
    
class Conv1dBlock(nn.Module):
    def __init__(self,ninp,noutp,kernel_size,stride=None,padding=0,Handle=True):
        super(Conv1dBlock,self).__init__()
        self.conv=nn.Conv1d(ninp,noutp,kernel_size,stride=stride,padding=padding)
        self.Handle=Handle
        self.norm=nn.BatchNorm1d(noutp)
        self.pool=nn.MaxPool1d(5,stride=1,padding=2)
        self.act=nn.ReLU()
        self.dropout=nn.Dropout()
    def forward(self,x):
        x=self.conv(x)
        if self.Handle:
            x=self.norm(x)
            x=self.pool(x)
            x=self.act(x)
            x=self.dropout(x)
        return x



In [9]:

class PPGNet(nn.Module):
    """
    网络结构定义
    """
    '''
    def __init__(self,ntr,nc):
        super(PPGNet,self).__init__()
#       ngf=opt.ngf # 
        self.ntr=ntr #每一段信号追踪的分片数
        self.nc=nc #中间卷积层的个数
        self.ConvBlocks=nn.ModuleList()
        self.conv1=nn.Conv2d(4,8,1)
        self.conv2=nn.Conv2d(8,16,(ntr,3))
        self.act2=nn.MaxPool2d(kernel_size=(1,2),stride=(1,2))
        for i in np.arange(nc):
            ConvBlock=Conv2dBlock(2**(i+4),2**(i+5),(1,3),(1,1))
            self.ConvBlocks.append(ConvBlock)
        self.conv3=nn.Conv2d(2**(nc+4),32,(1,1),(1,1))
        self.act3=nn.ELU()
        self.fc=nn.Sequential(
            nn.Linear(64,512),
            nn.ELU(),
            nn.Linear(512,1),
            nn.ELU()
        )
    '''
    def __init__(self,fmaps,input_dim1,input_dim2,hidden_dim):
            super(PPGNet,self).__init__()

            # fisrt CNN feature extractor
            self.hidden_dim=hidden_dim
            self.ParallelBlocks=nn.ModuleList()
            for idx,kernel in enumerate(fmaps,start=1):
                ConvBlock=Conv1dBlock(1,1,kernel,stride=2,Handle=False)
                self.ParallelBlocks.append(ConvBlock)
            # lstm feature extractor      
            self.lstm1=nn.LSTM(input_dim1,hidden_dim,2) #   
            self.lstm2=nn.LSTM(input_dim2,hidden_dim,2)# 
        
           # middle sequential blocks 
            self.ConvBlocks=nn.ModuleList()
            ConvBlock1=Conv1dBlock(1,32,40,stride=1)
            self.ConvBlocks.append(ConvBlock1)
#             for jdx in np.arange(8):
            ConvBlock2=Conv1dBlock(32,47,50,stride=1)
            self.ConvBlocks.append(ConvBlock2)
            
            # linear
            self.linear=nn.Sequential(
                nn.Linear(384*125,1024),
                nn.BatchNorm1d(1024),
                nn.ReLU(),
                
#                 nn.Linear(32768,16384),
#                 nn.BatchNorm1d(16384),
#                 nn.ReLU(),
                
#                 nn.Linear(32768,8192),
#                 nn.BatchNorm1d(8192),
#                 nn.ReLU(),
                
#                 nn.Linear(32768,1024),
#                 nn.BatchNorm1d(1024),
#                 nn.ReLU(),
                
                nn.Linear(1024,1),
                nn.ReLU()
            )
                
#             self.conv2=nn.Conv2d(8,16,(ntr,3))
#             self.act2=nn.MaxPool2d(kernel_size=(1,2),stride=(1,2))
#             for i in np.arange(nc):
#                 ConvBlock=Conv2dBlock(2**(i+4),2**(i+5),(1,3),(1,1))
#                 self.ConvBlocks.append(ConvBlock)
#             self.conv3=nn.Conv2d(2**(nc+4),32,(1,1),(1,1))
#             self.act3=nn.ELU()
#             self.fc=nn.Sequential(
#                 nn.Linear(64,512),
#                 nn.ELU(),
#                 nn.Linear(512,1),
#                 nn.ELU()
#             )

    def forward(self,x):
#         channel=x.size(1)
#         ipdb.set_trace()
#         print('1:',x.shape)
#         length=len(self.ParallelBlocks)
        ParallelOut=[]
        channelout=[]#每一个通道平行卷积后处理
        serialout=[]
        for chidx in np.arange(x.size(1)):
            for i in np.arange(len(self.ParallelBlocks)):
                Block=self.ParallelBlocks[i]
                InitOut=Block(x[:,chidx,:].unsqueeze(1))
                ParallelOut.append(InitOut)

            channelout=torch.cat(ParallelOut,2)
            ParallelOut=[]
            # sequential block
#             ipdb.set_trace()

            for chidy in np.arange(len(self.ConvBlocks)):
                SecondBlock=self.ConvBlocks[chidy]
                if chidy == 0:
                    SecondOut=SecondBlock(channelout)
                else:
                    SecondOut=SecondBlock(SecondOut)

                if chidy+1==len(self.ConvBlocks):
                    serialout.append(SecondOut)
        # 5 channel concat
    #   SecondIn=torch.cat(ParallelOut,1) # n*40*__
        
        
                
        
#         X=t.cat(ParallelOut,1)
#         X=self.conv1(X)
            
#         ipdb.set_trace()
#         for j in np.arange(len(self.ConvBlocks)):
#             Block=self.ConvBlocks[j]
#             X=Block(X)
#         ipdb.set_trace()
        hidden=(torch.zeros([2, x.size(0), self.hidden_dim],dtype=torch.double),
                torch.zeros([2, x.size(0), self.hidden_dim],dtype=torch.double))
        LstmOut,self.hidden=self.lstm1(x.view(x.size(1),-1,x.size(2)),hidden)
        
        concat1=torch.cat(serialout,1)
        LstmOut1=LstmOut.view(LstmOut.size(1),-1,LstmOut.size(2))
        concat=torch.cat((LstmOut1,concat1),1)
        
        hidden=(torch.zeros([2, concat.size(0), self.hidden_dim],dtype=torch.double),
                torch.zeros([2, concat.size(0), self.hidden_dim],dtype=torch.double))
        LstmOutp,self.hidden=self.lstm1(concat.view(concat.size(1),-1,concat.size(2)),hidden)
#         ipdb.set_trace()
        out=self.linear(LstmOutp.view(LstmOutp.size(1),-1))
        return out
#         x1=self.conv1(x)
#         print('2:',x.shape)
#         x=self.conv2(x)
#         print('3:',x.shape)
#         x=self.act2(x)
#         print('4:',x.shape)
#         pdb.set_trace()
#         for i in np.arange(self.nc):
#             ConvBlock=self.ConvBlocks[i]
#             x=ConvBlock(x)
#             print('{}:'.format(i+5),x.shape)
#         pdb.set_trace()
#         x=self.conv3(x)
#         x=self.act3(x)
#         x=x.view(x.size(0),1,-1)
#         out=self.fc(x)
#         return out.squeeze()
    
    def get_n_params(self):
        pp=0
        for p in list(self.parameters()):
            nn=1
            for s in list(p.size()):
                nn = nn*s
            pp += nn
        return pp 




In [10]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv2dBlock') != -1:
        print('Initializing weights of convresblock to 0.0, 0.02')
        for k, p in m.named_parameters():
            if 'weight' in k and 'conv' in k:
                p.data.normal_(0.0, 0.02)
    elif classname.find('Conv2d') != -1:
        print('Initialzing weight to 0.0, 0.02 for module: ', m)
        m.weight.data.normal_(0.0, 0.02)
        if hasattr(m, 'bias') and m.bias is not None:
            print('bias to 0 for module: ', m)
            m.bias.data.fill_(0)
    elif classname.find('Linear') != -1:
        print('Initializing FC weight to xavier uniform')
        nn.init.xavier_uniform_(m.weight.data)
def build_norm_layer(norm_type, param=None, num_feats=None):
    if norm_type == 'bnorm':
        return nn.BatchNorm2d(num_feats)
    elif norm_type == 'snorm':
        spectral_norm(param)
        return None
    elif norm_type is None:
        return None
    else:
        raise TypeError('Unrecognized norm type: ', norm_type)

In [11]:
kernels=[5,20,40,60,80]
model=PPGNet(kernels,125,125,125)
print('Total model parameters: ',model.get_n_params())

Total model parameters:  49737034


In [12]:
from torch.utils import data
import os
import scipy.io as sio
class PPGData(data.Dataset):
    def __init__(self,root):
        datas=os.listdir(root)
        self.totaldata=[os.path.join(root,data) for data in datas]
    def __getitem__(self,index):
#       timedata=np.zeros((4,7,1025))
#         ipdb.set_trace()
        ppgpath=self.totaldata[index]
        time=sio.loadmat(ppgpath)
        time=time['ppg']
        time=time.reshape(8,125)
        listslice=ppgpath.split('-')
        label=listslice[2][:-4]
        return time,label
    def __len__(self):
        return len(self.totaldata)
        


In [13]:

from torch.utils.data import DataLoader
filepath='/home/wcj/ReferenceProject/PPGnet/ppghr'
dataloader=PPGData(filepath)
# pdb.set_trace()
device = 'cpu'
if t.cuda.is_available:
#     device = t.device('cuda:0' if t.cuda.is_available() else 'cpu')
    device = 'cuda'
CUDA = (device == 'cuda')
print(device,CUDA)
# pdb.set_trace()
dloador=DataLoader(dataloader,batch_size=10,shuffle=True,num_workers=8,pin_memory=CUDA)

print(len(dataloader))
dataiter=iter(dloador)
sample,label=dataiter.next()

cuda True
1622


In [14]:
import torch.optim as optim
optimizer=optim.Adam(model.parameters(),lr=0.0001)

In [17]:
##################
#  lr=1e-5
##################
iteration=0
numepoch=0
dloador=DataLoader(dataloader,batch_size=10,shuffle=True,num_workers=8,pin_memory=CUDA)
optimizer=optim.Adam(model.parameters(),lr=0.01)
criterion=nn.L1Loss()
Net=model.double()
for epoch in np.arange(200):
    numepoch+=1
    running_loss=0.0
    for i,data in enumerate(dloador,0):
        
        iteration+=1
#         ipdb.set_trace()
        sample,labels=data
        labels=np.asarray(labels,dtype=float)
        labels=t.from_numpy(labels)
        sample.to(device)
        labels.to(device)
        labels=labels.view(labels.size(0),-1)

#         print(labels)
        optimizer.zero_grad()
#         sample=sample.double()
        outputs=Net(sample.double())
        loss=criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()
        if i%10==9:
            print('[%d,%5d] loss %f' %(epoch+1,i+1,running_loss/10))
            running_loss=0.0
print('finish training')
t.save(Net.state_dict(),'/home/wcj/ReferenceProject/PPGnet/checkpoints/0430Net-epoch-%d-iteration%d.pth'%(numepoch,iteration))

[1,   10] loss 114.826447
[1,   20] loss 63.512405
[1,   30] loss 39.749833
[1,   40] loss 41.679797
[1,   50] loss 31.475236
[1,   60] loss 27.918379
[1,   70] loss 26.658218
[1,   80] loss 31.830644
[1,   90] loss 30.356957
[1,  100] loss 24.866935
[1,  110] loss 24.057739
[1,  120] loss 28.117739
[1,  130] loss 23.935385
[1,  140] loss 25.033024
[1,  150] loss 28.318848
[1,  160] loss 22.006725
[2,   10] loss 24.693608
[2,   20] loss 21.320766
[2,   30] loss 19.278396
[2,   40] loss 19.730962
[2,   50] loss 18.081757
[2,   60] loss 19.896156
[2,   70] loss 26.025699
[2,   80] loss 26.202573
[2,   90] loss 20.298536
[2,  100] loss 21.959055
[2,  110] loss 19.594975
[2,  120] loss 17.566269
[2,  130] loss 17.684633
[2,  140] loss 16.103818
[2,  150] loss 20.078954
[2,  160] loss 20.343543
[3,   10] loss 20.398725
[3,   20] loss 16.114635
[3,   30] loss 19.313070
[3,   40] loss 18.034370
[3,   50] loss 18.534032
[3,   60] loss 17.725432
[3,   70] loss 17.384991
[3,   80] loss 17.535531

[21,   20] loss 11.733287
[21,   30] loss 12.170260
[21,   40] loss 12.847095
[21,   50] loss 11.080617
[21,   60] loss 9.764896
[21,   70] loss 10.937823
[21,   80] loss 9.249292
[21,   90] loss 13.049461
[21,  100] loss 11.415427
[21,  110] loss 13.379997
[21,  120] loss 10.604961
[21,  130] loss 9.090721
[21,  140] loss 8.264062
[21,  150] loss 12.846546
[21,  160] loss 9.792360
[22,   10] loss 12.689251
[22,   20] loss 12.162139
[22,   30] loss 10.490704
[22,   40] loss 10.454704
[22,   50] loss 11.918427
[22,   60] loss 12.610358
[22,   70] loss 12.856504
[22,   80] loss 10.877302
[22,   90] loss 10.100235
[22,  100] loss 12.851627
[22,  110] loss 11.239151
[22,  120] loss 11.871637
[22,  130] loss 13.507391
[22,  140] loss 14.503515
[22,  150] loss 13.117711
[22,  160] loss 12.354125
[23,   10] loss 11.102847
[23,   20] loss 11.501174
[23,   30] loss 11.849333
[23,   40] loss 12.656423
[23,   50] loss 10.818761
[23,   60] loss 10.301899
[23,   70] loss 10.272170
[23,   80] loss 1

[41,   20] loss 11.827612
[41,   30] loss 9.322524
[41,   40] loss 7.999043
[41,   50] loss 8.915833
[41,   60] loss 10.429987
[41,   70] loss 8.993097
[41,   80] loss 10.504537
[41,   90] loss 9.364591
[41,  100] loss 7.901154
[41,  110] loss 8.332145
[41,  120] loss 9.548835
[41,  130] loss 8.182590
[41,  140] loss 7.695733
[41,  150] loss 8.324579
[41,  160] loss 11.176021
[42,   10] loss 8.624420
[42,   20] loss 10.301882
[42,   30] loss 9.667352
[42,   40] loss 9.285037
[42,   50] loss 10.554078
[42,   60] loss 8.168023
[42,   70] loss 7.626632
[42,   80] loss 7.395015
[42,   90] loss 10.115675
[42,  100] loss 8.852825
[42,  110] loss 9.412097
[42,  120] loss 10.790533
[42,  130] loss 8.915978
[42,  140] loss 10.413921
[42,  150] loss 10.533427
[42,  160] loss 10.216706
[43,   10] loss 9.612207
[43,   20] loss 8.025902
[43,   30] loss 9.094998
[43,   40] loss 10.104631
[43,   50] loss 8.422684
[43,   60] loss 8.793226
[43,   70] loss 8.021216
[43,   80] loss 8.732087
[43,   90] lo

[61,   80] loss 5.983373
[61,   90] loss 8.918439
[61,  100] loss 7.920262
[61,  110] loss 8.872116
[61,  120] loss 7.592964
[61,  130] loss 9.683468
[61,  140] loss 8.218237
[61,  150] loss 7.283930
[61,  160] loss 8.221235
[62,   10] loss 7.571388
[62,   20] loss 7.731349
[62,   30] loss 9.293556
[62,   40] loss 8.405648
[62,   50] loss 9.204209
[62,   60] loss 7.628799
[62,   70] loss 6.733740
[62,   80] loss 8.011008
[62,   90] loss 8.077671
[62,  100] loss 7.277001
[62,  110] loss 7.247722
[62,  120] loss 7.785574
[62,  130] loss 7.486943
[62,  140] loss 7.786588
[62,  150] loss 8.243304
[62,  160] loss 9.291474
[63,   10] loss 9.383692
[63,   20] loss 9.224545
[63,   30] loss 8.563586
[63,   40] loss 7.547905
[63,   50] loss 9.155745
[63,   60] loss 8.912480
[63,   70] loss 6.159879
[63,   80] loss 9.056838
[63,   90] loss 7.471764
[63,  100] loss 8.249536
[63,  110] loss 10.032697
[63,  120] loss 8.941082
[63,  130] loss 7.496727
[63,  140] loss 7.194786
[63,  150] loss 6.992298

[81,  160] loss 7.816786
[82,   10] loss 9.522141
[82,   20] loss 9.185812
[82,   30] loss 7.071410
[82,   40] loss 6.287179
[82,   50] loss 6.282547
[82,   60] loss 6.825699
[82,   70] loss 6.891238
[82,   80] loss 7.957542
[82,   90] loss 7.422582
[82,  100] loss 6.876020
[82,  110] loss 7.441336
[82,  120] loss 7.191988
[82,  130] loss 8.525136
[82,  140] loss 6.637956
[82,  150] loss 6.401080
[82,  160] loss 8.098628
[83,   10] loss 5.327407
[83,   20] loss 7.414848
[83,   30] loss 7.534813
[83,   40] loss 7.282266
[83,   50] loss 6.685845
[83,   60] loss 8.969882
[83,   70] loss 8.175414
[83,   80] loss 10.020833
[83,   90] loss 7.345167
[83,  100] loss 7.058191
[83,  110] loss 6.920401
[83,  120] loss 7.687195
[83,  130] loss 7.681737
[83,  140] loss 6.936241
[83,  150] loss 8.221530
[83,  160] loss 7.327930
[84,   10] loss 6.875762
[84,   20] loss 8.245025
[84,   30] loss 9.567143
[84,   40] loss 7.622764
[84,   50] loss 7.718247
[84,   60] loss 9.059585
[84,   70] loss 7.952795

[102,   70] loss 8.573886
[102,   80] loss 9.065668
[102,   90] loss 6.929632
[102,  100] loss 7.574652
[102,  110] loss 7.772755
[102,  120] loss 5.898374
[102,  130] loss 6.703010
[102,  140] loss 6.956129
[102,  150] loss 6.250252
[102,  160] loss 6.935761
[103,   10] loss 7.288781
[103,   20] loss 7.668761
[103,   30] loss 7.400780
[103,   40] loss 6.277049
[103,   50] loss 8.791657
[103,   60] loss 8.661762
[103,   70] loss 8.002969
[103,   80] loss 6.588255
[103,   90] loss 7.507830
[103,  100] loss 6.329530
[103,  110] loss 6.639062
[103,  120] loss 7.146192
[103,  130] loss 6.684461
[103,  140] loss 6.159564
[103,  150] loss 8.185580
[103,  160] loss 6.516721
[104,   10] loss 5.910183
[104,   20] loss 6.938333
[104,   30] loss 7.869914
[104,   40] loss 6.906696
[104,   50] loss 8.811371
[104,   60] loss 7.641794
[104,   70] loss 6.190470
[104,   80] loss 5.544684
[104,   90] loss 6.299822
[104,  100] loss 7.725117
[104,  110] loss 7.272879
[104,  120] loss 6.940223
[104,  130] 

KeyboardInterrupt: 

In [18]:
dloador=DataLoader(dataloader,batch_size=100,shuffle=True,num_workers=8,pin_memory=CUDA)
optimizer=optim.Adam(model.parameters(),lr=0.001)
criterion=nn.L1Loss()
Net=model.double()
for epoch in np.arange(50):
    numepoch+=1
    running_loss=0.0
    for i,data in enumerate(dloador,0):
        
        iteration+=1
#         ipdb.set_trace()
        sample,labels=data
        labels=np.asarray(labels,dtype=float)
        labels=t.from_numpy(labels)
        sample.to(device)
        labels.to(device)
        labels=labels.view(labels.size(0),-1)

#         print(labels)
        optimizer.zero_grad()
#         sample=sample.double()
        outputs=Net(sample.double())
        loss=criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()
#         if i%10==9:
        print('[%d,%5d] loss %f' %(epoch+1,i+1,running_loss))
        running_loss=0.0
print('finish training')
t.save(Net.state_dict(),'/home/wcj/ReferenceProject/PPGnet/checkpoints/0501Net-epoch-%d-iteration%d.pth'%(numepoch,iteration))

[1,    1] loss 9.633750
[1,    2] loss 10.868176
[1,    3] loss 8.799723
[1,    4] loss 8.057599
[1,    5] loss 9.887116
[1,    6] loss 8.868018
[1,    7] loss 8.018954
[1,    8] loss 8.674002
[1,    9] loss 11.006958
[1,   10] loss 8.130774
[1,   11] loss 8.201711
[1,   12] loss 8.026394
[1,   13] loss 7.844780
[1,   14] loss 8.032592
[1,   15] loss 7.374043
[1,   16] loss 8.098491
[1,   17] loss 7.544830
[2,    1] loss 8.448002
[2,    2] loss 7.455944
[2,    3] loss 7.795868
[2,    4] loss 7.627667
[2,    5] loss 9.042590
[2,    6] loss 7.449772
[2,    7] loss 9.121800
[2,    8] loss 8.756799
[2,    9] loss 7.902645
[2,   10] loss 6.771464
[2,   11] loss 8.405308
[2,   12] loss 7.653183
[2,   13] loss 8.430473
[2,   14] loss 7.559853
[2,   15] loss 7.347771
[2,   16] loss 7.339405
[2,   17] loss 10.559498
[3,    1] loss 8.362894
[3,    2] loss 7.804941
[3,    3] loss 8.408666
[3,    4] loss 6.678304
[3,    5] loss 7.920934
[3,    6] loss 8.073853
[3,    7] loss 7.949444
[3,    8] los

[20,   12] loss 5.163803
[20,   13] loss 4.661687
[20,   14] loss 5.549439
[20,   15] loss 6.642075
[20,   16] loss 6.075464
[20,   17] loss 9.372734
[21,    1] loss 5.818889
[21,    2] loss 5.433639
[21,    3] loss 6.212713
[21,    4] loss 5.225773
[21,    5] loss 6.133182
[21,    6] loss 5.301756
[21,    7] loss 6.540111
[21,    8] loss 6.726844
[21,    9] loss 6.381639
[21,   10] loss 5.161125
[21,   11] loss 6.705449
[21,   12] loss 5.548649
[21,   13] loss 5.036759
[21,   14] loss 5.899362
[21,   15] loss 6.165009
[21,   16] loss 5.674161
[21,   17] loss 5.051189
[22,    1] loss 5.284895
[22,    2] loss 5.390654
[22,    3] loss 5.421516
[22,    4] loss 6.155219
[22,    5] loss 6.667243
[22,    6] loss 6.259992
[22,    7] loss 6.092795
[22,    8] loss 5.550002
[22,    9] loss 5.527746
[22,   10] loss 5.834141
[22,   11] loss 5.316856
[22,   12] loss 5.423941
[22,   13] loss 5.234830
[22,   14] loss 5.036290
[22,   15] loss 7.124423
[22,   16] loss 5.550985
[22,   17] loss 8.630186


[39,   17] loss 5.817623
[40,    1] loss 4.272336
[40,    2] loss 6.127992
[40,    3] loss 5.042657
[40,    4] loss 4.461900
[40,    5] loss 5.119647
[40,    6] loss 6.269674
[40,    7] loss 5.277754
[40,    8] loss 4.596263
[40,    9] loss 6.395602
[40,   10] loss 4.929570
[40,   11] loss 5.195877
[40,   12] loss 4.689006
[40,   13] loss 5.297588
[40,   14] loss 5.197390
[40,   15] loss 5.674514
[40,   16] loss 6.054297
[40,   17] loss 6.243636
[41,    1] loss 4.892457
[41,    2] loss 5.926201
[41,    3] loss 4.415909
[41,    4] loss 5.487018
[41,    5] loss 4.557688
[41,    6] loss 5.989112
[41,    7] loss 6.008581
[41,    8] loss 5.729324
[41,    9] loss 5.314916
[41,   10] loss 5.704570
[41,   11] loss 5.709727
[41,   12] loss 5.077909
[41,   13] loss 5.578302
[41,   14] loss 5.312421
[41,   15] loss 7.600875
[41,   16] loss 5.047894
[41,   17] loss 6.706929
[42,    1] loss 5.842254
[42,    2] loss 5.334108
[42,    3] loss 5.721776
[42,    4] loss 5.187110
[42,    5] loss 5.377655


In [19]:
###############################
# 模型实时测试 在12组数据(训练)上
###############################
testfilepath='/home/wcj/ReferenceProject/PPGnet/ppghr'
Testdataloader=PPGData(testfilepath)
testloader=DataLoader(
Testdataloader,
    batch_size=10,
    shuffle=True,
    num_workers=2
)

# for i in np.arange(7):
sub=0
total=0
with t.no_grad():
    for data in testloader:
        samples,labels=data
        labels=np.asarray(labels,dtype=float)
        labels=t.from_numpy(labels)
        labels=labels.view(labels.size(0),-1)
        outputs=model(samples)
    #         _,predicted=t.max(outputs,1)# 在矩阵的第一个维度上
        total+=labels.size(0)
#         ipdb.set_trace()
        sub+=abs(outputs-labels).sum()
    #         print('++++')
    print(' %d个样本计算MAE: %.6f'%(total,sub/total))

 1622个样本计算MAE: 8.453950


In [20]:
###############################
# 模型实时测试 在10组数据上
###############################

testfilepath='/home/wcj/ReferenceProject/PPGnet/testdata'
Testdataloader=PPGData(testfilepath)
testloader=DataLoader(
Testdataloader,
    batch_size=10,
    shuffle=True,
    num_workers=2
)

# for i in np.arange(7):
sub=0
total=0
with t.no_grad():
    for data in testloader:
        samples,labels=data
        labels=np.asarray(labels,dtype=float)
        labels=t.from_numpy(labels)
        labels=labels.view(labels.size(0),-1)
        outputs=model(samples)
    #         _,predicted=t.max(outputs,1)# 在矩阵的第一个维度上
        total+=labels.size(0)
#         ipdb.set_trace()
        sub+=abs(outputs-labels).sum()
    #         print('++++')
    print(' %d个样本计算MAE: %.6f'%(total,sub/total))

 1328个样本计算MAE: 28.039368


In [21]:
########################
# leave one
########################

# Net=PPGNet(kernels,125,125,125).eval()
# Net.double()
# Net.load_state_dict(t.load('/home/wcj/ReferenceProject/PPGnet/checkpoints/0430Net_epoch200-iteration32600.pth'))

testfilepath='/home/wcj/ReferenceProject/PPGnet/12thData'
Testdataloader=PPGData(testfilepath)
testloader=DataLoader(
Testdataloader,
    batch_size=10,
    shuffle=True,
    num_workers=2
)

# for i in np.arange(7):
sub=0
total=0
with t.no_grad():
    for data in testloader:
        samples,labels=data
        labels=np.asarray(labels,dtype=float)
        labels=t.from_numpy(labels)
        labels=labels.view(labels.size(0),-1)
        outputs=model(samples)
    #         _,predicted=t.max(outputs,1)# 在矩阵的第一个维度上
        total+=labels.size(0)
#         ipdb.set_trace()
        sub+=abs(outputs-labels).sum()
    #         print('++++')
    print(' %d个样本计算MAE: %.6f'%(total,sub/total))

 146个样本计算MAE: 10.356282
